In [1]:
%cd drive/MyDrive/collab_sandbox/cerf_text_classification/
!ls

/content/drive/MyDrive/collab_sandbox/cerf_text_classification
cerf_parsing.ipynb	datasets
CERF_texts_parsing	learnamericanenglishonline_links.csv
CERF_texts_parsing.zip	learnamericanenglishonline_links.gsheet


In [ ]:
# !unzip CERF_texts_parsing.zip

# learnamericanenglishonline

In [10]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
import os
import requests
import pprint 
"""
A1 => https://www.learnamericanenglishonline.com/Reading/Blue_Level_Reading/Blue_Level_Reading_Room.html
A2 => https://www.learnamericanenglishonline.com/Reading/Red_Level_Reading/Red_Level_Reading_Room.html
B1 => https://www.learnamericanenglishonline.com/Reading/Yellow_Level_Reading/Yellow_Level_Reading_Room.html
B2 => https://www.learnamericanenglishonline.com/Reading/Green_Level_Reading/Green_Level_Reading_Room.html
"""
pp = pprint.PrettyPrinter(indent=4)
# url = "https://www.learnamericanenglishonline.com/Reading/Green_Level_Reading/Green_Level_Reading_Room.html"
# page = requests.get(url)
# soup = BeautifulSoup(page.content, 'html.parser')
# links_list = soup.find(
#     'td', attrs={'class': 'bodyText'}).findAll('a', href=True)
# bad_chars = ["../", 'http']
# for item in links_list:
#   for char in bad_chars:
#     href = str(item['href'])
#     cond = sum([int(char in href) for char in bad_chars])
#     # print(cond)
#     if cond == 0:
#       print(href)


class ScrapWebsite:
    def __init__(self):
        self.domain_name = "https://www.learnamericanenglishonline.com/"
        self.name = 'learnamericanenglishonline'
        self.column_names_list = ['link', 'level', 'name']
        self.column_names_texts = ['source_text',
                                   'level', 'link', 'name', 'metadata']
        self.pages_list_urls = {
            'A1': ["https://www.learnamericanenglishonline.com/Reading/Blue_Level_Reading/", "Blue_Level_Reading_Room.html"],
            'A2': ['https://www.learnamericanenglishonline.com/Reading/Red_Level_Reading/', "Red_Level_Reading_Room.html"],
            'B1': ["https://www.learnamericanenglishonline.com/Reading/Yellow_Level_Reading/", "Yellow_Level_Reading_Room.html"],
            'B2': ["https://www.learnamericanenglishonline.com/Reading/Green_Level_Reading/", "Green_Level_Reading_Room.html"],
        }

    def parse_pages_urls(self, page_list_url, level):
        page = requests.get(page_list_url)
        soup = BeautifulSoup(page.content, 'html5lib')

        links_list = soup.find('td', attrs={'class': 'bodyText'}).findAll('a', href=True)
        bad_chars = ["../", 'http']

        rows = []
        # get hrefs
        for item in links_list:
          for char in bad_chars:
            href = str(item['href'])
            cond = sum([int(char in href) for char in bad_chars])
            # print(cond)
            if cond == 0:
              # print(href)

              article_link = self.pages_list_urls[level][0] + href
              row = {
                  'link': article_link,
                  'level': level,
                  'name': self.name
              }
              rows.append(row)
        return rows

    def get_all_links(self):
        dataframe = pd.DataFrame(columns=self.column_names_list)

        for level, list_url in self.pages_list_urls.items():
            list_url = list_url[0] + list_url[1]
            # print()
            texts_links = self.parse_pages_urls(list_url, level)
            for item in texts_links:
                dataframe = dataframe.append(item, ignore_index=True)

        dataframe.to_csv(f"{self.name}_links.csv", index=False)
        print('OK. Links parsed.')

    def get_all_texts(self):
        dataframe_links = pd.read_csv("continuingstudies_links.csv")
        dataframe_texts = pd.DataFrame(columns=self.column_names_texts)
        data_len = len(dataframe_links)
        for i in range(data_len):
            # print(row)
            row = dataframe_links.iloc[i]
            level = row['level']
            link = row['link']
            new_row = self.scrap_one_page(link, level)
            dataframe_texts = dataframe_texts.append(
                new_row, ignore_index=True)
        dataframe_texts.to_csv(f'{self.name}_texts.csv', index=False)
        print('OK. Texts Parsed.')

    def scrap_one_page(self, url, level):
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        title = soup.find('td', attrs={'class': 'pageName'})
        title = title.getText()
        page = soup.find('article')
        # text, audio_link = self.remove_trash(text)
        print(str(page))
        # print(page)
        # row = {
        #     'level': level,
        #     'source_text': text,
        #     'link': url,
        #     'name': self.name,
        #     'metadata': "{'audio_link': %s}" % audio_link
        # }
        return page

    def remove_trash(self, text=None):
        new_text = str(text)
        regex_array = [
            r"<sup><strong>.<\/strong><\/sup>",
            r"(<([^>]+)>)",
            r"Your browser does not support the audio element, so here's a link to the mp3:",
            r"[’”“]",
            r"\n",
            r"(Credits:.*)"
        ]
        for regex_str in regex_array:
            search_line = re.compile(regex_str)
            new_text = search_line.sub("", new_text).strip()

        audio_link = ""
        search_line = re.compile(r"(https.*\.mp3)")
        new_audio_link = search_line.findall(new_text)
        if len(new_audio_link) > 0:
            audio_link = new_audio_link[0]
        new_text = search_line.sub("", new_text).strip()

        return new_text, audio_link


if __name__ == '__main__':
  site_scrapper = ScrapWebsite()
  # site_scrapper.get_all_links()
	# site_scrapper.get_all_texts()
  link = "https://www.learnamericanenglishonline.com/Reading/Blue_Level_Reading/6_Jacob_and_Matt_are_studying.html"

site_scrapper.scrap_one_page(link, "a1")


<article class="post-1419 page type-page status-publish" id="post-1419" itemscope="" itemtype="https://schema.org/CreativeWork"><div class="inside-article"><div class="entry-content" itemprop="text"><table border="0" cellpadding="2" cellspacing="0" class="table"><tr><td class="pageName" colspan="3">6. Jacob and Matt are studying English online together.</td></tr><tr><td class="subHeader"></td></tr></table></div></div></article>


<article class="post-1419 page type-page status-publish" id="post-1419" itemscope="" itemtype="https://schema.org/CreativeWork"><div class="inside-article"><div class="entry-content" itemprop="text"><table border="0" cellpadding="2" cellspacing="0" class="table"><tr><td class="pageName" colspan="3">6. Jacob and Matt are studying English online together.</td></tr><tr><td class="subHeader"></td></tr></table></div></div></article>